# Confuison Matrix

* `True positive` = in binary classification, when the model should predict 1 and the truth is 1
* `True negative` = in binary classification, when the model should predict 0 and the truth is 0
* `False positive` = in binary classification, when the model should predict 1 and the truth is 0
* `False negative` = in binary classification, when the model should predict 0 and the truth is 1

In [ ]:
from sklearn.datasets import make_circles

# Make 1000 examples
n_samples = 1000

# Create circles
X, y = make_circles(n_samples,
                    noise=0.03,
                    random_state=42)

# Slipt into training data and test data
X_train, y_train = X[:800], y[:800]
X_test, y_test = X[800:], y[800:]

In [ ]:
import tensorflow as tf

# Set the random seed
tf.ramdom.seed(42)

# 1.Create the model usingthe Sequential API
model_1 = tf.keras.Sequential([
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(4, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid'),
])

#2. Compile the model
model_1.compile(loss= "binary_crossentropy",
                optimizers=tf.keras.optimizers.Adam(lr=0.02),
                matrics=["accuracy"])

# 3. FIt the model
model_1.fit(X_train, y_train, epochs=20)

In [ ]:
# Create a confusion matrix
from sklearn.metrics import confusion_matrix

# Make predictions
y_preds =  model_1.predict(X_test)

# Create confusion matrix
confusion_matrix(y_test, y_preds)

Looks like our predictions array has come out in prediction propability form. The standard output from the sigmoid (or softmax) activation functions. So now we should convert them.